In [16]:
using Connectomes
using ADNIDatasets
using CSV, DataFrames
using DrWatson: projectdir
using Distributions
using Serialization
using DelimitedFiles
using MCMCChains
using CairoMakie
using Turing

In [2]:
include(projectdir("functions.jl"))
include(projectdir("adni/inference/inference-preamble.jl"));

In [3]:

pst = deserialize(projectdir("adni/new-chains/local-fkpp/length-free/pst-taupos-4x2000.jls"));
pst2 = deserialize(projectdir("adni/new-chains/local-fkpp/length-free/pst-tauneg-4x2000.jls"));
pst3 = deserialize(projectdir("adni/new-chains/local-fkpp/length-free/pst-abneg-4x2000.jls"));

In [4]:
function get_diff(d)
    d[:,end] .- d[:,1]
end

get_diff (generic function with 1 method)

In [5]:
taupos_data = data[tau_pos]
taupos_suvr = calc_suvr.(taupos_data)
taupos_subdata = [normalise(sd, u0, cc) for sd in taupos_suvr]

taupos_vols = get_vol.(taupos_data)
taupos_total_vol_corrected = [v ./ sum(v, dims=1) for v in taupos_vols]
taupos_atr = [v ./ v[:,1] for v in taupos_total_vol_corrected];

In [24]:
qc_data = ADNIDataset(posdf, dktnames; min_scans=3, qc=false)

tauneg_data = qc_data[tau_neg]
tauneg_subsuvr = calc_suvr.(tauneg_data)
_tauneg_subdata = [normalise(sd, u0, cc) for sd in tauneg_subsuvr];

tauneg_blsd = [sd .- u0 for sd in _tauneg_subdata]
tauneg_nonzerosubs = findall(x -> sum(x) < 2, [sum(sd, dims=1) .== 0 for sd in tauneg_blsd])
tauneg_subdata = _tauneg_subdata[tauneg_nonzerosubs];

tauneg_vols = get_vol.(tauneg_data)[tauneg_nonzerosubs]
tauneg_total_vol_corrected = [v ./ sum(v, dims=1) for v in tauneg_vols]
tauneg_atr = [v ./ v[:,1] for v in tauneg_total_vol_corrected];

In [22]:
abneg_data = ADNIDataset(negdf, dktnames; min_scans=3, qc=false)

abneg_subsuvr = calc_suvr.(abneg_data)
_abneg_subdata = [normalise(sd, u0, cc) for sd in abneg_subsuvr]

abneg_blsd = [sd .- u0 for sd in _abneg_subdata]
abneg_nonzerosubs = findall(x -> sum(x) < 2, [sum(sd, dims=1) .== 0 for sd in abneg_blsd])

abneg_subdata = _abneg_subdata[abneg_nonzerosubs
]
abneg_vols = get_vol.(abneg_data)[abneg_nonzerosubs]
abneg_total_vol_corrected = [v ./ sum(v, dims=1) for v in abneg_vols]
abneg_atr = [v ./ v[:,1] for v in abneg_total_vol_corrected];

In [20]:
taupos_meanpst = mean(pst)
taupos_production = [taupos_meanpst["α[$i]", :mean] for i in 1:57]
taupos_transport  = [taupos_meanpst["ρ[$i]", :mean] for i in 1:57];

tauneg_meanpst = mean(pst2)
tauneg_production = [tauneg_meanpst["α[$i]", :mean] for i in 1:37]
tauneg_transport  = [tauneg_meanpst["ρ[$i]", :mean] for i in 1:37];

abneg_meanpst = mean(pst3)
abneg_production = [abneg_meanpst["α[$i]", :mean] for i in 1:58]
abneg_transport  = [abneg_meanpst["ρ[$i]", :mean] for i in 1:58];

In [21]:
cols = Makie.wong_colors() 

taupos_total_initial_suvr = [sum(s, dims=1)[end] for s in taupos_subdata]
tauneg_total_initial_suvr = [sum(s, dims=1)[end] for s in tauneg_subdata]
abneg_total_initial_suvr = [sum(s, dims=1)[end] for s in abneg_subdata]


taupos_suvr_diff = mean.([get_diff(sd[mtl,:]) for sd in taupos_subdata])
taupos_vol_diff = mean.([get_diff(sd[mtl,:]) for sd in taupos_atr])

tauneg_suvr_diff = mean.([get_diff(sd[mtl,:]) for sd in tauneg_subdata])
tauneg_vol_diff = mean.([get_diff(sd[mtl,:]) for sd in tauneg_atr])

abneg_suvr_diff = mean.([get_diff(sd[mtl,:]) for sd in abneg_subdata])
abneg_vol_diff = mean.([get_diff(sd[mtl,:]) for sd in abneg_atr])

begin
    f = Figure(size=(1000,500))
    ax = Axis(f[1,1])
    scatter!(taupos_suvr_diff, taupos_vol_diff, color=(cols[1], 0.75), markersize=10)
    scatter!(tauneg_suvr_diff, tauneg_vol_diff, color=(cols[2], 0.75), markersize=10)
    scatter!(abneg_suvr_diff, abneg_vol_diff, color=(cols[3], 0.75), markersize=10)

    ax = Axis(f[1,2])
    scatter!(taupos_production, taupos_vol_diff, color=(cols[1], 0.75), markersize=10)
    scatter!(tauneg_production, tauneg_vol_diff, color=(cols[2], 0.75), markersize=10)
    scatter!(abneg_production, abneg_vol_diff, color=(cols[3], 0.75), markersize=10)

    f
end


DimensionMismatch: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 37 and 39

In [10]:
begin
    f = Figure(size=(1000,500))
    ax = Axis(f[1,1])
    scatter!(taupos_production, taupos_total_initial_suvr, color=(cols[1], 0.75), markersize=15)
    scatter!(tauneg_production, tauneg_total_initial_suvr, color=(cols[2], 0.75), markersize=15)
    scatter!(abneg_transport, abneg_total_initial_suvr, color=(cols[3], 0.75), markersize=15)
    ax = Axis(f[1,2])
    scatter!(taupos_transport, taupos_total_initial_suvr, color=(cols[1], 0.75), markersize=15)
    scatter!(tauneg_transport, tauneg_total_initial_suvr, color=(cols[2], 0.75), markersize=15)
    # scatter!(abneg_transport, abneg_total_initial_suvr, color=(cols[3], 0.75), markersize=15)
    f
end

DimensionMismatch: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 37 and 39